<a href="https://colab.research.google.com/github/dieko95/AlgoTrading/blob/develop/AlgoTrading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AlgoTrading

## Download Alt Data (Tweets)

https://pypi.org/project/GetOldTweets3/



```
python Exporter.py --query @RedHat --since 2014-01-01 --until 2019-01-15 --maxtweets 5000000
```



In [0]:
!pip install GetOldTweets3

In [0]:
import pandas as pd
import GetOldTweets3 as got


In [0]:
maxTweets=20 

tweetCriteria = got.manager.TweetCriteria().setQuerySearch('@RedHat')\
                                           .setSince("2014-01-01")\
                                           .setUntil("2018-01-15")\
                                           .setMaxTweets(20)




tweet = got.manager.TweetManager.getTweets(tweetCriteria)

#dir(tweet)

# tweet.text

In [0]:
# dates = []
# text = []

# for i in range(10):
  
#   got.manager.TweetCriteria().setQuerySearch('@RedHat')\
#                                            .setSince("2014-01-01")\
#                                            .setUntil("2018-01-15")\
#                                            .setMaxTweets(i)
#   dates.append()
  

TypeError: ignored

In [0]:
# got.manager.TweetManager.getTweets(tweetCriteria)[0].text

dates = []
text = []
for i in range(20):
  print('tweet',i)
  tweet = got.manager.TweetManager.getTweets(tweetCriteria)[i]
  dates.append(tweet.date)
  text.append(tweet.text)
   
# print(dates)  
#   print(got.manager.TweetManager.getTweets(tweetCriteria).text[i]

tweet 0
tweet 1
tweet 2
tweet 3
tweet 4
tweet 5
tweet 6
tweet 7
tweet 8
tweet 9
tweet 10
tweet 11
tweet 12
tweet 13
tweet 14
tweet 15
tweet 16
tweet 17
tweet 18
tweet 19


In [0]:
# got.manager.TweetManager.getTweets(tweetCriteria)[0].text


tweetsDf = pd.DataFrame({'dates':dates, 'text':text})




## Sentiment Analysis with Vader 

In [0]:
tweetsDf = pd.read_csv('redhatprototype.csv')

In [0]:
# !pip install vaderSentiment

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


analyzer = SentimentIntensityAnalyzer()

tweet = []
number_favourites = []
retweets = []
vs_compound = []
vs_pos = []
vs_neu = []
vs_neg = []
timeStamp = []

for i in range(tweetsDf.shape[0]):
    tweet.append(tweetsDf.loc[i]['text'])
    number_favourites.append(tweetsDf.loc[i]['favorites'])
    retweets.append(tweetsDf.loc[i]['retweets'])
    timeStamp.append(tweetsDf.loc[i]['date'])
    vs_compound.append(analyzer.polarity_scores(tweetsDf.loc[i]['text'])['compound'])
    vs_pos.append(analyzer.polarity_scores(tweetsDf.loc[i]['text'])['pos'])
    vs_neu.append(analyzer.polarity_scores(tweetsDf.loc[i]['text'])['neu'])
    vs_neg.append(analyzer.polarity_scores(tweetsDf.loc[i]['text'])['neg'])

In [137]:
from pandas import Series, DataFrame

twitter_df = DataFrame({'Tweet': tweet,
                        'Favourites': number_favourites,
                        'retweets' : retweets,
                        'timeStamp' : timeStamp,
                        'Compound': vs_compound,
                        'Positive': vs_pos,
                        'Neutral': vs_neu,
                        'Negative': vs_neg})
twitter_df = twitter_df[['timeStamp','Tweet', 'Favourites', 'retweets','Compound',
                         'Positive', 'Neutral', 'Negative']]

twitter_df

,timeStamp,Tweet,Favourites,retweets,Compound,Positive,Neutral,Negative
0,2019-01-14 18:24,Tomorrow I am teaching a course on Buildah wit...,29,16,0.0000,0.000,1.000,0.000
1,2019-01-14 18:23,"Rob Szumski, Principal Product Manager at Red ...",3,2,-0.4939,0.041,0.853,0.106
2,2019-01-14 18:00,Wondering how 3rd party apps can be managed at...,0,0,0.6369,0.157,0.843,0.000
3,2019-01-14 17:27,Easily check real-time status of the #RedHat C...,4,4,0.6486,0.209,0.791,0.000
4,2019-01-13 12:42,In my two teams that I'm a part of at @redhat ...,7,0,0.5994,0.162,0.782,0.057
5,2019-01-13 11:50,@RedHat Cert Prep #RHCSA 3: Konfiguration von ...,2,2,0.0000,0.000,1.000,0.000
6,2019-01-13 11:50,I’ll be in Westford and working on the custome...,1,0,0.0000,0.000,1.000,0.000
7,2019-01-13 11:50,"@RedHat Cert Prep RHCSA 5: Bereitstellung, Kon...",2,2,0.0000,0.000,1.000,0.000
8,2019-01-13 11:50,@RedHat Cert Prep RHCSA 6: Verwaltung von Benu...,2,2,0.0000,0.000,1.000,0.000
9,2019-01-13 11:49,@RedHat Cert Prep RHCSA 7: Sicherheitsmanageme...,2,2,0.0000,0.000,1.000,0.000
